In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.chrome.options import Options

import tqdm

import numpy as np
import pandas as pd

import re

from datetime import datetime

import time
import os

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [6]:
ruta = os.getcwd()

In [7]:
ruta

'/Users/cevalenciam/Library/CloudStorage/OneDrive-Personal/SmartBuy/smartbuy/web-scraper-tottus'

In [8]:
url = 'https://tottus.falabella.com/tottus-cl/page/horario_tiendas'

In [9]:
def get_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    chrome_options.add_experimental_option("prefs",prefs)
    s = ChromeService("chromedriver")
    driver = webdriver.Chrome(service=s, options=chrome_options)
    driver.maximize_window()
    return driver

In [10]:
driver = get_driver()

#Vamos a la dirección web de la página objetivo
driver.get(url)

list_direcciones = []

#Obtener las tiendas
try:
    wait(driver, 64).until(ec.presence_of_element_located((By.XPATH, "/html/body/div[@id='__next']/div/div[@class='container-fluid container-xl']/div[@class='row BaseTemplate-module_main-row__1dr8i']/div[@class='AccordeonItems-module_container__0Tqsb pb-3 col-md-12']/div[@class='AccordeonItems-module_container-desktop__sN1qw']")))
    regiones = driver.find_elements("xpath", "/html/body/div[@id='__next']/div/div[@class='container-fluid container-xl']/div[@class='row BaseTemplate-module_main-row__1dr8i']/div[@class='AccordeonItems-module_container__0Tqsb pb-3 col-md-12']/div[@class='AccordeonItems-module_container-desktop__sN1qw']")
    
    for reg in tqdm.tqdm(regiones):
        
        #scroll para visualizar
        driver.execute_script("arguments[0].scrollIntoView();", reg)

        region = reg.find_elements("xpath", "div[@class='AccordeonItems-module_title-container__qQktq']/div[@class='AccordeonItems-module_title-component-container__4Bx-S']/div[@class='AccordeonItems-module_title-component-title-wrapper__xQHlp']/span[@class='AccordeonItems-module_title-accordion__355b5']")[0].text
        
        comunas = reg.find_elements("xpath", "div[@class='AccordeonItems-module_accordeon-desktop__UEDBa']/ul/li[@class='AccordeonItems-module_card-item__8URy3']/div[@class='Accordion-module_toggler__hOkhR']")

        for com in comunas:
             
            #scroll para visualizar
            driver.execute_script("arguments[0].scrollIntoView();", com)
            
            comu = com.find_elements("xpath", "div[@class='Accordion-module_list-item__sXhcF']/div[@class='Accordion-module_title-item__-WbJb']")[0]
            
            comu.click()

            comuna = comu.text
            
            direccion = com.find_elements("xpath", "div[@class='Accordion-module_content__eS3R7']/div[@class='Accordion-module_content-text__OwYKK']/span")[0].text

            list_direcciones.append([region, comuna, direccion])

except:
        print("Regiones no encontradas")

driver.close()

100%|██████████| 9/9 [00:11<00:00,  1.27s/it]


In [11]:
list_direcciones[-2:]

[['Región del Biobío',
  'Los Ángeles',
  'Mendoza 535 local H-100\nHorario \nLunes a Domingo 08:30 a 21:30 hrs\nUbicada dentro de Centro Comercial'],
 ['Región del Biobío',
  'Los Ángeles Alemania',
  'Av. Alemania 831\nHorario \nLunes a Domingo 08:30 a 21:30 hrs\nUbicada dentro de Centro Comercial']]

In [13]:
list_direcciones_final = []

driver = get_driver()

for r, c, dir in tqdm.tqdm(list_direcciones):

    direccion = dir[:dir.index("\n")] + ', ' + c + ", " + r + ", Chile, Tottus"

    driver.get("https://www.google.com/maps/search/"+direccion)

    time.sleep(3)

    try:
        wait(driver, 4).until(ec.visibility_of_element_located((By.XPATH, "html/body/div[@id='app-container']/div[@id='content-container']/div[@id='QA0Szd']/div/div[@class='XltNde tTVLSc']/div[@class='w6VYqd']/div[@class='bJzME tTVLSc']/div[@class='k7jAl lJ3Kh miFGmb']/div[@tabindex='-1']/div[@class='aIFcqe']/div[@class='m6QErb WNBkOb ']/div[@class='TIHn2 ']/div[@class='tAiQdd']/div[@class='lMbq3e']")))

        coordenadas = driver.find_element(By.CSS_SELECTOR, 'meta[itemprop=image]').get_attribute('content')

        latitud = float(coordenadas[coordenadas.find('-'):coordenadas.find("%")])
        longitud =float(coordenadas[coordenadas.find('-', coordenadas.find("-")+1):coordenadas.find("&")])

        wait(driver, 4).until(ec.visibility_of_element_located((By.XPATH, "html/body/div[@id='app-container']/div[@id='content-container']/div[@id='QA0Szd']/div/div[@class='XltNde tTVLSc']/div[@class='w6VYqd']/div[@class='bJzME tTVLSc']/div[@class='k7jAl lJ3Kh miFGmb']/div[@tabindex='-1']/div[@class='aIFcqe']/div[@class='m6QErb WNBkOb ']/div[@class='ZKCDEc']/div[@class='RZ66Rb FgCUCc']/button[@class='aoRNLd kn2E5e NMjTrf lvtCsd ']/img")))
        imagen = driver.find_element(By.XPATH, "html/body/div[@id='app-container']/div[@id='content-container']/div[@id='QA0Szd']/div/div[@class='XltNde tTVLSc']/div[@class='w6VYqd']/div[@class='bJzME tTVLSc']/div[@class='k7jAl lJ3Kh miFGmb']/div[@tabindex='-1']/div[@class='aIFcqe']/div[@class='m6QErb WNBkOb ']/div[@class='ZKCDEc']/div[@class='RZ66Rb FgCUCc']/button[@class='aoRNLd kn2E5e NMjTrf lvtCsd ']/img").get_attribute('src')

        comentario = 'Dirección encontrada'
    
    except:

        try:
            wait(driver, 4).until(ec.element_to_be_clickable((By.CLASS_NAME, "hfpxzc")))
            boton = driver.find_element(By.CLASS_NAME, "hfpxzc")
            boton.click()

            time.sleep(3)

            coordenadas = driver.find_element(By.CSS_SELECTOR, 'meta[itemprop=image]').get_attribute('content')

            latitud = float(coordenadas[coordenadas.find('-'):coordenadas.find("%")])
            longitud =float(coordenadas[coordenadas.find('-', coordenadas.find("-")+1):coordenadas.find("&")])

            wait(driver, 4).until(ec.visibility_of_element_located((By.XPATH, "html/body/div[@id='app-container']/div[@id='content-container']/div[@id='QA0Szd']/div/div[@class='XltNde tTVLSc']/div[@class='w6VYqd']/div[@class='bJzME Hu9e2e tTVLSc']/div[@class='k7jAl lJ3Kh miFGmb']/div[@tabindex='-1']/div[@class='aIFcqe']/div[@class='m6QErb WNBkOb ']/div[@class='m6QErb DxyBCb kA9KIf dS8AEf ']/div[@class='ZKCDEc']/div[@class='RZ66Rb FgCUCc']/button[@class='aoRNLd kn2E5e NMjTrf lvtCsd ']/img")))
            imagen = driver.find_element(By.XPATH, "html/body/div[@id='app-container']/div[@id='content-container']/div[@id='QA0Szd']/div/div[@class='XltNde tTVLSc']/div[@class='w6VYqd']/div[@class='bJzME Hu9e2e tTVLSc']/div[@class='k7jAl lJ3Kh miFGmb']/div[@tabindex='-1']/div[@class='aIFcqe']/div[@class='m6QErb WNBkOb ']/div[@class='m6QErb DxyBCb kA9KIf dS8AEf ']/div[@class='ZKCDEc']/div[@class='RZ66Rb FgCUCc']/button[@class='aoRNLd kn2E5e NMjTrf lvtCsd ']/img").get_attribute('src')

            comentario = 'Más de una dirección encontrada, toma la primera'
            
            print(comentario)

        except:
            
            comentario = "Dirección no encontrada"
            print(comentario)
        
            latitud = np.nan
            longitud = np.nan

            imagen = ''
            
            driver.get("https://www.google.com/maps")

    list_direcciones_final.append([r, c, dir, imagen, latitud, longitud, comentario])

driver.close()

  8%|▊         | 6/72 [00:34<06:00,  5.46s/it]

Dirección no encontrada


 28%|██▊       | 20/72 [02:00<04:44,  5.47s/it]

Dirección no encontrada


 57%|█████▋    | 41/72 [04:03<02:45,  5.34s/it]

...

 90%|█████████ | 65/72 [06:47<00:41,  5.87s/it]

Dirección no encontrada


 92%|█████████▏| 66/72 [07:03<00:54,  9.00s/it]

Dirección no encontrada


100%|██████████| 72/72 [07:47<00:00,  6.49s/it]


In [18]:
list_direcciones_final

[['Región Metropolitana',
  'Alameda',
  "Av Libertador Bernardo O'Higgins 3470, Santiago\nHorario\nLunes a Domingo 08:30 a 21:30 hrs\nUbicada dentro de Centro Comercial",
  'https://lh5.googleusercontent.com/p/AF1QipNBTsaCtsVTP0ZsWUcWxQe8LL0S3xOWW9DMjfxV=w426-h240-k-no',
  -33.4524185,
  -70.6823731,
  'Dirección encontrada'],
 ['Región Metropolitana',
  'Buín',
  'San Martín 174, Buin\nHorario\nLunes a Domingo 08:30 a 21:30 hrs',
  'https://lh5.googleusercontent.com/p/AF1QipObZ2ineBycAcBBAEdHaU9BhQ6ZQFtY-ESbLYIA=w408-h306-k-no',
  -33.73176,
  -70.7348277,
  'Dirección encontrada'],
 ['Región Metropolitana',
  'Catedral',
  'Catedral 1850, Santiago\nHorario\nLunes a Viernes 08:00 a 21:30 hrs\nSabado y Domingo 08:30 a 21:30 hrs\nUbicada dentro de Centro Comercial',
  'https://lh5.googleusercontent.com/p/AF1QipNgslfiVDYbz82t3BzYlvQzhwfiKCsYXlRvL2k=w408-h544-k-no',
  -33.4388556,
  -70.6630539,
  'Dirección encontrada'],
 ['Región Metropolitana',
  'Ciudad Empresarial',
  'Av. del Parqu

In [15]:
df_tiendas_tottus = pd.DataFrame(list_direcciones_final, columns=['region', 'comuna', 'direccion', 'imagen', 'latitud', 'longitud', 'comentario'])

In [16]:
df_tiendas_tottus.head(3)

,region,comuna,direccion,imagen,latitud,longitud,comentario
0,Región Metropolitana,Alameda,"Av Libertador Bernardo O'Higgins 3470, Santiag...",https://lh5.googleusercontent.com/p/AF1QipNBTs...,-33.452419,-70.682373,Dirección encontrada
1,Región Metropolitana,Buín,"San Martín 174, Buin\nHorario\nLunes a Domingo...",https://lh5.googleusercontent.com/p/AF1QipObZ2...,-33.731760,-70.734828,Dirección encontrada
2,Región Metropolitana,Catedral,"Catedral 1850, Santiago\nHorario\nLunes a Vier...",https://lh5.googleusercontent.com/p/AF1QipNgsl...,-33.438856,-70.663054,Dirección encontrada


In [17]:
actual_date = str(datetime.now())[0:10]

df_tiendas_tottus.to_excel(ruta + f'/df_tiendas_tottus.xlsx'.format(actual_date), index=0)